In [12]:
import os
import json
import numpy as np
import pandas as pd
from random import shuffle
import matplotlib.pyplot as plt
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/ESTSoft/Data/

/content/drive/.shortcut-targets-by-id/1p3ZVbysNHbjR_R-DsFNNFAsoss96Ugz9/Data


In [7]:
TRAIN_JSON_PATH = '/content/final_data/label/train/'
VALID_JSON_PATH = '/content/final_data/label/val/'

TRAIN_IMG_PATH = '/content/final_data/img/train/'
VALID_IMG_PATH = '/content/final_data/img/val/'

TRAIN_SEGMENT_PATH = '/content/final_data/segmentation/train/'
VALID_SEGMENT_PATH = '/content/final_data/segmentation/val/'

In [8]:
with open (TRAIN_JSON_PATH + "train_anger.json", "r", encoding='euc-kr') as f:
    anger_data = json.load(f)
with open (TRAIN_JSON_PATH + "train_happy.json", "r", encoding='euc-kr') as f:
    happy_data = json.load(f)
with open (TRAIN_JSON_PATH + "train_panic.json", "r", encoding='euc-kr') as f:
    panic_data = json.load(f)
with open (TRAIN_JSON_PATH + "train_sadness.json", "r", encoding='euc-kr') as f:
    sadness_data = json.load(f)

In [9]:
# JSON 데이터를 DataFrame으로 변환
anger_df = pd.DataFrame(anger_data)
happy_df = pd.DataFrame(happy_data)
panic_df = pd.DataFrame(panic_data)
sadness_df = pd.DataFrame(sadness_data)

In [25]:
# 각 DataFrame을 리스트에 저장
dfs = [anger_df, happy_df, panic_df, sadness_df]

# DataFrame 합치기
train_df = pd.concat(dfs, ignore_index=True)
train_df = train_df.sort_values(by='filename')
train_df.head()

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
3398,007b10b45d424e7876e73b6b66d62d603c6ef2ee886761...,여,20,일반인,당황,숙박 및 거주공간,"{'boxes': {'maxX': 1888.9304000000002, 'maxY':...","{'boxes': {'maxX': 1888.8512363165685, 'maxY':...","{'boxes': {'maxX': 1888.9304, 'maxY': 1413.411..."
643,00ceb5eb09f1ee2f4144cbda7fd5561fb1b0252310231e...,여,30,일반인,분노,공공시설/종교/의료시설,"{'boxes': {'maxX': 1808.2333000000008, 'maxY':...","{'boxes': {'maxX': 1792.0642309314867, 'maxY':...","{'boxes': {'maxX': 1808.2332999999996, 'maxY':..."
1264,00hj57bc2be60739b1b9f99316a3a11de5febe993b0938...,여,40,일반인,분노,행사/사무공간,"{'boxes': {'maxX': 1843.9142463679543, 'maxY':...","{'boxes': {'maxX': 1866.0104193116026, 'maxY':...","{'boxes': {'maxX': 1852.6880000000003, 'maxY':..."
5596,00skd6f03c1bc8e3f618fb80b7863679bf684363a4d013...,여,20,전문인,슬픔,문화재 및 유적지,"{'boxes': {'maxX': 1624.9164, 'maxY': 1418.914...","{'boxes': {'maxX': 1621.3714458715597, 'maxY':...","{'boxes': {'maxX': 1624.9163999999998, 'maxY':..."
4749,00yucd996df0c12e2dcac91e0a41d77a35181e5ff4cce6...,남,40,전문인,슬픔,실외 자연환경,"{'boxes': {'maxX': 2250.07, 'maxY': 1605.0603,...","{'boxes': {'maxX': 2250.2525267323604, 'maxY':...","{'boxes': {'maxX': 2250.1939980518337, 'maxY':..."


In [31]:
anger_df = anger_df.sort_values(by='filename')
anger_df.reset_index(drop=True, inplace=True)
anger_df

,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A,annot_B,annot_C
0,00ceb5eb09f1ee2f4144cbda7fd5561fb1b0252310231e...,여,30,일반인,분노,공공시설/종교/의료시설,"{'boxes': {'maxX': 1808.2333000000008, 'maxY':...","{'boxes': {'maxX': 1792.0642309314867, 'maxY':...","{'boxes': {'maxX': 1808.2332999999996, 'maxY':..."
1,00hj57bc2be60739b1b9f99316a3a11de5febe993b0938...,여,40,일반인,분노,행사/사무공간,"{'boxes': {'maxX': 1843.9142463679543, 'maxY':...","{'boxes': {'maxX': 1866.0104193116026, 'maxY':...","{'boxes': {'maxX': 1852.6880000000003, 'maxY':..."
2,013320f6c6aa0752eb2933111106c9bcbfaaf62b416862...,여,50,일반인,분노,교통/이동수단(엘리베이터 포함),"{'boxes': {'maxX': 2383.9530546875003, 'maxY':...","{'boxes': {'maxX': 2382.5422949120093, 'maxY':...","{'boxes': {'maxX': 2378.7810263028277, 'maxY':..."
3,03nf73f4eb25c42388b21c54304c809be3fed9c8815eb2...,남,30,전문인,분노,공공시설/종교/의료시설,"{'boxes': {'maxX': 2065.9468000000006, 'maxY':...","{'boxes': {'maxX': 2057.5193786270265, 'maxY':...","{'boxes': {'maxX': 2044.2389975229373, 'maxY':..."
4,05s25ed5288ff5e3f2172d3c4c21b368113be833decb26...,여,20,일반인,분노,도심 환경,"{'boxes': {'maxX': 2982.1381999999994, 'maxY':...","{'boxes': {'maxX': 3035.3788594813122, 'maxY':...","{'boxes': {'maxX': 3029.6966827517267, 'maxY':..."
...,...,...,...,...,...,...,...,...,...
1495,zx1z7a73d6976eee78347847db5180b094fe70e7aa0920...,여,20,전문인,분노,상업시설/점포/시장,"{'boxes': {'maxX': 1939.2924, 'maxY': 1493.827...","{'boxes': {'maxX': 1939.2923999999998, 'maxY':...","{'boxes': {'maxX': 1916.2299061397478, 'maxY':..."
1496,zy326c174c014be2ded498c447f94dc091ff4fd0c142dd...,여,20,전문인,분노,도심 환경,"{'boxes': {'maxX': 1874.2059000000002, 'maxY':...","{'boxes': {'maxX': 1827.0454028550246, 'maxY':...","{'boxes': {'maxX': 1821.95895589011, 'maxY': 1..."
1497,zybhf86ef77e020261b33724557a6f881876051e2e6789...,여,30,전문인,분노,문화재 및 유적지,"{'boxes': {'maxX': 2363.174799999999, 'maxY': ...","{'boxes': {'maxX': 2363.1747999999993, 'maxY':...","{'boxes': {'maxX': 2363.174799999999, 'maxY': ..."
1498,zyct94aba1ee0d1e6292f6f56ecdba6d29a5b6e1ca43b6...,남,20,전문인,분노,오락/공연시설,"{'boxes': {'maxX': 2672.7248362320383, 'maxY':...","{'boxes': {'maxX': 2691.9801024658796, 'maxY':...","{'boxes': {'maxX': 2661.0699312862025, 'maxY':..."


In [42]:
# npz 파일 불러오기
npzfile = np.load('/content/final_data/segmentation/train/train_anger.npz')
file_names = npzfile.files

# 디렉토리 내 이미지 파일 경로
directory_path = '/content/final_data/img/train/anger'
image_files = sorted(os.listdir(directory_path))

# Green 부분만 추출하여 이미지로 저장하는 함수(머리카락, 안경 등등 제외하고 얼굴만 추출)
def save_green_region_as_image(mask, original_image, save_directory, file_name):
    try:
        # Green 부분 추출
        green_mask = (mask == 3)
        green_part = np.zeros_like(original_image, dtype=np.uint8)
        green_part[green_mask] = original_image[green_mask]

        # 좌표 가져오기
        maxX = anger_df['annot_A'][i]['boxes']['maxX']
        maxY = anger_df['annot_A'][i]['boxes']['maxY']
        minX = anger_df['annot_A'][i]['boxes']['minX']
        minY = anger_df['annot_A'][i]['boxes']['minY']

        # 이미지 자르기
        green_part = green_part[int(minY):int(maxY), int(minX):int(maxX)]

        # 이미지 저장 경로 설정
        save_path = os.path.join(save_directory, file_name)

        # 이미지 저장
        cv2.imwrite(save_path, green_part)
    except Exception as e:
        print(f"Error occurred while processing {file_name}: {e}")

# Green 부분만을 원본 이미지에서 추출하여 이미지로 저장
for i, file_name in enumerate(file_names[:]):
    if file_name in image_files:
        mask = npzfile[file_name]
        original_image_path = os.path.join(directory_path, file_name)
        original_image = cv2.imread(original_image_path)
        save_directory = '/content/anger_crop/'
        save_file_name = f'anger{i+1}.jpg'
        save_green_region_as_image(mask, original_image, save_directory, save_file_name)
    else:
        print(f"Image file for {file_name} not found.")

Error occurred while processing anger405.jpg: OpenCV(4.8.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Error occurred while processing anger495.jpg: OpenCV(4.8.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

